<a href="https://colab.research.google.com/github/tarunsha009/LLM-Learning/blob/main/gpt2_finetuned_cs_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00


In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 21.3 MB/s eta 0:00:00


In [3]:
from pypdf import PdfReader

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        pdf_reader = PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

book1 = extract_text_from_pdf("/content/Building Microservices - Designing Fine-Grained Systems.pdf")
book2 = extract_text_from_pdf("/content/Building Microservices.pdf")
book3 = extract_text_from_pdf("/content/Clean Architecture A Craftsman Guide to Software Structure and Design.pdf")
book4 = extract_text_from_pdf("/content/Clean.Code.A.Handbook.of.Agile.Software.Craftsmanship.pdf")
book5 = extract_text_from_pdf("/content/Guru's SDF (1).pdf")


combined_text = book1 + book2 + book3 + book4 + book5

with open("cs_books.txt", "w") as f:
    f.write(combined_text)

In [4]:
print(len(combined_text))

2397657


In [5]:
import re

def clean_text(text):
  text = re.sub(r'\n', ' ', text)
  text = re.sub(r'[^\w\s.,;:!?]', '', text)
  return text

cleaned_text = clean_text(combined_text)


In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

chunks = [cleaned_text[i:i+1000] for i in range(0, len(cleaned_text), 1000)]

In [14]:
from datasets import Dataset

dataset = Dataset.from_dict({"text": chunks})


def tokenize_function(examples):
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        return_tensors="pt",
        padding="max_length"
    )
    # Set labels = input_ids (shifted for next-token prediction)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].clone()
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(tokenized_datasets[0].keys())  # Should show: input_ids, attention_mask, labels

Map:   0%|          | 0/2356 [00:00<?, ? examples/s]

dict_keys(['text', 'input_ids', 'attention_mask', 'labels'])


In [15]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer

model = GPT2LMHeadModel.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="gpt2-cs-books",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    fp16=True,
    save_steps=10_000,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,1.777700
1000,1.416500


Step,Training Loss
500,1.777700
1000,1.416500
1500,1.338100


TrainOutput(global_step=1767, training_loss=1.4846698823593132, metrics={'train_runtime': 472.1095, 'train_samples_per_second': 14.971, 'train_steps_per_second': 3.743, 'total_flos': 1846812082176000.0, 'train_loss': 1.4846698823593132, 'epoch': 3.0})

In [19]:
def generate_text(prompt, model, tokenizer, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,       # Increase for more randomness (try 0.7-1.0)
        top_k=50,              # Sample from top 50 likely next tokens
        top_p=0.95,            # Nucleus sampling: picks from top tokens covering 95% probability
        repetition_penalty=1.2,  # Penalize repeated phrases (values >1.0 reduce repetition)
        do_sample=True,        # Enable sampling (required for temperature/top_k/top_p)
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example prompt about Clean Code
prompt = "The key principles of Clean Code are:"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The key principles of Clean Code are: Use clean code that is clear and concise. This means a consistent, readable and structured rule for the use case in which it happens to be used; this makes sense as part ﬁrst thing we do on site development teams or small businesses using an automated system such systems with minimal downtime!  Avoid making too many promises about what will happen when you run out your hands free after just five minutes at work ! A good example would come from one simple


In [20]:
prompt = "Explain the key principles of Clean Code in computer programming:"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Explain the key principles of Clean Code in computer programming: clean code, consistency and compactness.  5 How do you make sure that your program is cleaner? In this chapter we will discuss some simple ideas to help us keep our programs as small or smaller than possible so they can be more compacted into manageable parts at a time when it s convenient for each individual project on its own system; how much longer should I have an office environment before writing my software? 3 Keep Your Software Small enough


In [21]:
model.save_pretrained("gpt2-finetuned-cs-books")
tokenizer.save_pretrained("gpt2-finetuned-cs-books")

('gpt2-finetuned-cs-books/tokenizer_config.json',
 'gpt2-finetuned-cs-books/special_tokens_map.json',
 'gpt2-finetuned-cs-books/vocab.json',
 'gpt2-finetuned-cs-books/merges.txt',
 'gpt2-finetuned-cs-books/added_tokens.json',
 'gpt2-finetuned-cs-books/tokenizer.json')

In [22]:
def generate_high_quality_text(prompt, model, tokenizer, max_length=150):
    # Encode the input with attention mask
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(model.device)

    # Generate text with better parameters
    output = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,           # Balanced creativity
        top_k=40,                  # Focus on top probable tokens
        top_p=0.9,                 # Nucleus sampling
        repetition_penalty=1.3,     # Reduce repetition
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        early_stopping=True
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

# Usage:
prompt = "Explain the key principles of Clean Code in computer programming:"
print(generate_high_quality_text(prompt, model, tokenizer))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Explain the key principles of Clean Code in computer programming: Write clean code. Make it easy for other people to understand what you do, and make sure that they understand your craft! Learn from others who are doing well too ! Clean coding is a discipline where we should strive together as one team with our shared goals but also work on different ends . If any part or cause will be eliminated by someone else using an existing tool rather than something better at making us smarter
